## Commit

Running the cell below copies any local source code modifications into the FlashAttention install directory on the FlashAttention docker image, and then commits the Docker image.

In [625]:
!./build_commit.sh

"docker kill" requires at least 1 argument.
See 'docker kill --help'.

Usage:  docker kill [OPTIONS] CONTAINER [CONTAINER...]

Kill one or more running containers
Untagged: flash:flattened_live
Deleted: sha256:d5c093f2bff4b3ccc4e829bb7bc17512935bf1f857826d7efd747ac23809687a
Creating flash-attention_flashattention_run ... 
ting flash-attention_flashattention_run ... done

torch.__version__  = 1.14.0a0+410ce96


running install
running bdist_egg
running egg_info
writing flash_attn.egg-info/PKG-INFO
writing dependency_links to flash_attn.egg-info/dependency_links.txt
writing requirements to flash_attn.egg-info/requires.txt
writing top-level names to flash_attn.egg-info/top_level.txt
reading manifest file 'flash_attn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
/usr/local/lib/python3.8/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/l

byte-compiling build/bdist.linux-x86_64/egg/flash_attn/flash_attention.py to flash_attention.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/gpt.py to gpt.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/llama.py to llama.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/__init__.py to __init__.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/gpt_neox.py to gpt_neox.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/gptj.py to gptj.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/bert.py to bert.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/vit.py to vit.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/opt.py to opt.cpython-38.pyc
creating stub loader for flash_attn_cuda.cpython-38-x86_64-linux-gnu.so
byte-compiling build/bdist.linux-x86_64/egg/flash_attn_cuda.py t

## Testing

In [ ]:
--capture=tee-sys

In [627]:
!echo "pytest --capture=tee-sys tests/test_flash_attn_lse.py"  | docker-compose run flashattention 

Creating flash-attention_flashattention_run ... 
ting flash-attention_flashattention_run ... done============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/workspace
plugins: rerunfailures-10.3, xdoctest-1.0.2, hypothesis-5.35.1, xdist-3.1.0, shard-0.1.2, hydra-core-1.3.1
collected 3 items
Running 3 items in this shard

tests/test_flash_attn_lse.py ...                                         [100%]

============================== 3 passed in 1.74s ===============================
